In [1]:
import hnswlib
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import numpy as np
from keras.datasets import mnist
from time import perf_counter
from ast import literal_eval

In [2]:
name = 'mnist-784'
runs = 1
metric = 'euclidean'
queries = 1000
nameFull = name + '-' + metric + '-true-labels.xlsx'
datasetTrainImages, datasetTestImages, _ = get_ann_benchmark_data(name)

d:\Учеба\Назарбаев Университет\Masters\raAndThesis\PavelBraslavski\annSearchComparisons/datasets/mnist-784.hdf5


<string>:90: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


trainDataset :  (60000, 784)
testDataset :  (10000, 784)


<string>:91: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
<string>:92: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


***Create HNSW Index***

In [3]:
def createIndex(indexMethod, datasetImages):
    d = datasetImages.shape[1] 
    num_elements = datasetImages.shape[0] 
    time_start = perf_counter()
    index = indexMethod(space = 'l2', dim = d)
    index.init_index(max_elements = num_elements, ef_construction = 200, M = 16)
    ids = np.arange(num_elements)
    index.add_items(datasetImages, ids)
    index.set_ef(120)
    time_end = perf_counter()
    totalTime = (time_end - time_start)
    return (index, totalTime)

In [4]:
(minBuildTime, maxBuildTime, indexedStruct) = createIndexNumerous(createIndex, hnswlib.Index, datasetTrainImages, runs)
print('minBuildTime : ', minBuildTime)
print('maxBuildTime : ', maxBuildTime)

minBuildTime :  30.548
maxBuildTime :  30.548


In [5]:
indexes = []
distances = []
def measureTime(par, indexes, distances, datasetImages):
    totalTime = 0
    for i in range(par) : 
        xq = datasetImages[i:i+1].astype('float32') # Use the first image as the query vector
        time_start = perf_counter()
        index, distance = indexedStruct.knn_query(xq, k = 100)
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        indexes.append(index[0])
        distances.append(np.sqrt(distance[0]))
    return np.round(totalTime, 3)

In [6]:
(minSearchTime, maxSearchTime, indexes, distances) = measureTimeNumerous(measureTime, runs, queries, datasetTestImages)
print('minSearchTime : ', minSearchTime)
print('maxSearchTime : ', maxSearchTime)

minSearchTime :  1.162
maxSearchTime :  1.162


In [7]:
indexes = np.array(indexes)
distances = np.round(np.array(distances).astype(float), 4)

In [8]:
print('indexes : ', indexes.shape)
print('distances : ', distances.shape)

indexes :  (1000, 100)
distances :  (1000, 100)


In [9]:
path = '../datasets/'+nameFull
(trueIndexes, trueDistances) = readDB(path)

trueIndexes :  (1000, 100)
trueDistances :  (1000, 100)


In [10]:
amount = 10
compareElems(amount, indexes, distances, trueIndexes, trueDistances)

53843 || 53843
676.584 || 676.584
38620 || 38620
793.9868 || 793.9868
16186 || 16186
862.6766 || 862.6766
27059 || 27059
864.5039 || 864.5039
47003 || 47003
894.7 || 894.7
14563 || 14563
909.7043 || 909.7043
44566 || 44566
917.6323 || 917.6323
15260 || 15260
921.6241 || 921.6241
40368 || 40368
922.147 || 922.147
36395 || 36395
943.4972 || 943.4972


In [11]:
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.01)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.1)

Recall@1: 0.8984
Recall@1.01: 0.9999
Recall@1.1: 1.0000


1.0